In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time("2023-08-14 00:00:00Z", scale='utc')
end = Time("2023-08-19 00:00:00Z", scale='utc')
eStop = await client.select_time_series('lsst.sal.ATDome.logevent_emergencyStop', ['*'], start, end)
state = await client.select_time_series('lsst.sal.ATDome.logevent_summaryState', ['summaryState'], start, end)


In [ ]:
print(len(state),len(eStop))

In [ ]:
eStop

In [ ]:
for i in range(11, 17):
    print(i, ptg.iloc[i]['pointingModelTermValues'])

In [ ]:
az['actualPosition'].plot()

In [ ]:
file

In [ ]:
client.select_time_series?

In [ ]:
start = Time("2023-03-17 20:00:00Z", scale='utc')
end = Time("2023-03-17 20:05:00Z", scale='utc')
test1 = await client.select_time_series('lsst.sal.ESS.accelerometerPSD', \
                                            ['*'],  start, end, index=202)


In [ ]:
len(test1)

In [ ]:
len(test1.columns)

In [ ]:
test1['salIndex']

In [ ]:
start = Time("2023-03-17 20:00:00Z", scale='utc')
end = Time("2023-03-17 20:05:00Z", scale='utc')
test2 = await client.select_time_series('lsst.sal.ESS.accelerometerPSD', \
                                            ['*'],  start, end, index=104)


In [ ]:
len(test2)

In [ ]:
len(test2.columns)

In [ ]:
test2['salIndex']